In [94]:
import pickle
X_hr,y_stress,y_participant,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr.p','rb'))

In [95]:
X_hr = X_hr.reshape(-1,30,1)

In [96]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input
from keras.models import Model, Sequential
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder

In [97]:
train_participant = y_participant.copy()
train_participant = OneHotEncoder().fit_transform(train_participant.reshape(-1,1)).todense()

In [98]:
np.unique(y_participant),train_participant.shape

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 (42928, 15))

In [99]:
train_x, test_x, train_y, test_y,participant_ids_train, participant_ids_test = train_test_split(X_hr,
                                                            train_participant,
                                                            y_participant,
                                                            test_size = 0.2,
                                                            random_state=41,
                                                            stratify=y_participant)

In [100]:
train_x, val_x, train_y, val_y, participant_ids_train, participant_ids_val  = train_test_split(train_x,
                                                            train_y,
                                                            participant_ids_train,
                                                            test_size = 0.1,
                                                            random_state=41,
                                                            stratify=participant_ids_train)

In [101]:
train_x.shape,test_x.shape,val_x.shape,train_y.shape,test_y.shape,val_y.shape,participant_ids_train.shape

((30907, 30, 1),
 (8586, 30, 1),
 (3435, 30, 1),
 (30907, 15),
 (8586, 15),
 (3435, 15),
 (30907,))

In [102]:
def get_model(input_shape=(30,1),act='relu',loss="categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes,activation='sigmoid',kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt,metrics=['acc'])
    return model

In [103]:
n_classes = len(np.unique(y_participant))
model = get_model(input_shape=(30,1),n_classes=n_classes)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_31 (Conv1D)           (None, 30, 100)           1100      
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 30, 100)           100100    
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 15, 100)           0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 15, 100)           400       
_________________________________________________________________
dropout_22 (Dropout)         (None, 15, 100)           0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 15, 100)           100100    
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 7, 100)           

In [104]:
from keras.models import load_model
filepath = '../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5'

In [105]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

Train on 30907 samples, validate on 3435 samples
Epoch 1/400
30907/30907 [==============================] - 4s 141us/step - loss: 2.5649 - acc: 0.1754 - val_loss: 2.7319 - val_acc: 0.0952

Epoch 00001: val_loss improved from inf to 2.73190, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 2/400
30907/30907 [==============================] - 3s 91us/step - loss: 2.4100 - acc: 0.2087 - val_loss: 2.4764 - val_acc: 0.1424

Epoch 00002: val_loss improved from 2.73190 to 2.47642, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 3/400
30907/30907 [==============================] - 3s 92us/step - loss: 2.3545 - acc: 0.2169 - val_loss: 2.3346 - val_acc: 0.2207

Epoch 00003: val_loss improved from 2.47642 to 2.33459, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 4/400
30907/30907 [==============================] - 3s 92us/step - loss: 2.3221 - acc: 0.2238 - val_loss: 2.3253 - val_acc: 0.2277

Epoch 00004: val_loss i

Epoch 36/400
30907/30907 [==============================] - 3s 91us/step - loss: 1.4980 - acc: 0.4875 - val_loss: 1.5574 - val_acc: 0.4655

Epoch 00036: val_loss did not improve from 1.54388
Epoch 37/400
30907/30907 [==============================] - 3s 91us/step - loss: 1.4733 - acc: 0.4923 - val_loss: 1.4569 - val_acc: 0.4978

Epoch 00037: val_loss improved from 1.54388 to 1.45686, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 38/400
30907/30907 [==============================] - 3s 92us/step - loss: 1.4614 - acc: 0.4971 - val_loss: 1.5841 - val_acc: 0.4544

Epoch 00038: val_loss did not improve from 1.45686
Epoch 39/400
30907/30907 [==============================] - 3s 92us/step - loss: 1.4313 - acc: 0.5110 - val_loss: 1.6639 - val_acc: 0.4361

Epoch 00039: val_loss did not improve from 1.45686
Epoch 40/400
30907/30907 [==============================] - 3s 92us/step - loss: 1.4036 - acc: 0.5176 - val_loss: 1.5872 - val_acc: 0.4620

Epoch 00040: val_loss 

30907/30907 [==============================] - 3s 92us/step - loss: 0.9088 - acc: 0.6944 - val_loss: 0.8553 - val_acc: 0.7261

Epoch 00073: val_loss improved from 0.89999 to 0.85535, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 74/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.8908 - acc: 0.7011 - val_loss: 0.8437 - val_acc: 0.7109

Epoch 00074: val_loss improved from 0.85535 to 0.84369, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 75/400
30907/30907 [==============================] - 3s 93us/step - loss: 0.8799 - acc: 0.7040 - val_loss: 0.8234 - val_acc: 0.7284

Epoch 00075: val_loss improved from 0.84369 to 0.82342, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 76/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.8756 - acc: 0.7038 - val_loss: 0.7882 - val_acc: 0.7444

Epoch 00076: val_loss improved from 0.82342 to 0.78822, saving model to ../mod

30907/30907 [==============================] - 3s 92us/step - loss: 0.6236 - acc: 0.7923 - val_loss: 0.5019 - val_acc: 0.8440

Epoch 00110: val_loss improved from 0.52432 to 0.50189, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 111/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.6242 - acc: 0.7898 - val_loss: 0.4980 - val_acc: 0.8445

Epoch 00111: val_loss improved from 0.50189 to 0.49797, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 112/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.6148 - acc: 0.7943 - val_loss: 0.5386 - val_acc: 0.8311

Epoch 00112: val_loss did not improve from 0.49797
Epoch 113/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.6156 - acc: 0.7941 - val_loss: 0.5583 - val_acc: 0.8215

Epoch 00113: val_loss did not improve from 0.49797
Epoch 114/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.5943 - acc: 0.

30907/30907 [==============================] - 3s 91us/step - loss: 0.4614 - acc: 0.8460 - val_loss: 0.5194 - val_acc: 0.8329

Epoch 00148: val_loss did not improve from 0.35394
Epoch 149/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.4789 - acc: 0.8391 - val_loss: 0.3447 - val_acc: 0.8905

Epoch 00149: val_loss improved from 0.35394 to 0.34470, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 150/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.4794 - acc: 0.8392 - val_loss: 0.3681 - val_acc: 0.8841

Epoch 00150: val_loss did not improve from 0.34470
Epoch 151/400
30907/30907 [==============================] - 3s 93us/step - loss: 0.4710 - acc: 0.8430 - val_loss: 0.4030 - val_acc: 0.8670

Epoch 00151: val_loss did not improve from 0.34470
Epoch 152/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.4695 - acc: 0.8427 - val_loss: 0.4223 - val_acc: 0.8632

Epoch 00152: val_loss did not i

...

30907/30907 [==============================] - 3s 91us/step - loss: 0.3202 - acc: 0.8934 - val_loss: 0.2336 - val_acc: 0.9325

Epoch 00227: val_loss did not improve from 0.18953
Epoch 228/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.3132 - acc: 0.8954 - val_loss: 0.2977 - val_acc: 0.8984

Epoch 00228: val_loss did not improve from 0.18953
Epoch 229/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.3154 - acc: 0.8949 - val_loss: 0.1996 - val_acc: 0.9432

Epoch 00229: val_loss did not improve from 0.18953
Epoch 230/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.3259 - acc: 0.8918 - val_loss: 0.3043 - val_acc: 0.9057

Epoch 00230: val_loss did not improve from 0.18953
Epoch 231/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.3129 - acc: 0.8956 - val_loss: 0.1952 - val_acc: 0.9424

Epoch 00231: val_loss did not improve from 0.18953
Epoch 232/400
30907/30907 [===========================

30907/30907 [==============================] - 3s 92us/step - loss: 0.2767 - acc: 0.9075 - val_loss: 0.1620 - val_acc: 0.9502

Epoch 00267: val_loss did not improve from 0.15399
Epoch 268/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.2747 - acc: 0.9079 - val_loss: 0.1600 - val_acc: 0.9496

Epoch 00268: val_loss did not improve from 0.15399
Epoch 269/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.2748 - acc: 0.9074 - val_loss: 0.1677 - val_acc: 0.9482

Epoch 00269: val_loss did not improve from 0.15399
Epoch 270/400
30907/30907 [==============================] - 3s 92us/step - loss: 0.2706 - acc: 0.9098 - val_loss: 0.1955 - val_acc: 0.9392

Epoch 00270: val_loss did not improve from 0.15399
Epoch 271/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2791 - acc: 0.9068 - val_loss: 0.1724 - val_acc: 0.9461

Epoch 00271: val_loss did not improve from 0.15399
Epoch 272/400
30907/30907 [===========================

30907/30907 [==============================] - 3s 90us/step - loss: 0.2523 - acc: 0.9164 - val_loss: 0.1918 - val_acc: 0.9389

Epoch 00308: val_loss did not improve from 0.13021
Epoch 309/400
30907/30907 [==============================] - 3s 90us/step - loss: 0.2508 - acc: 0.9161 - val_loss: 0.1328 - val_acc: 0.9578

Epoch 00309: val_loss did not improve from 0.13021
Epoch 310/400
30907/30907 [==============================] - 3s 90us/step - loss: 0.2464 - acc: 0.9189 - val_loss: 0.1823 - val_acc: 0.9441

Epoch 00310: val_loss did not improve from 0.13021
Epoch 311/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2467 - acc: 0.9179 - val_loss: 0.1601 - val_acc: 0.9525

Epoch 00311: val_loss did not improve from 0.13021
Epoch 312/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2368 - acc: 0.9217 - val_loss: 0.1576 - val_acc: 0.9523

Epoch 00312: val_loss did not improve from 0.13021
Epoch 313/400
30907/30907 [===========================

Epoch 349/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2265 - acc: 0.9261 - val_loss: 0.1173 - val_acc: 0.9665

Epoch 00349: val_loss did not improve from 0.11232
Epoch 350/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2235 - acc: 0.9248 - val_loss: 0.1184 - val_acc: 0.9613

Epoch 00350: val_loss did not improve from 0.11232
Epoch 351/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2198 - acc: 0.9284 - val_loss: 0.1975 - val_acc: 0.9380

Epoch 00351: val_loss did not improve from 0.11232
Epoch 352/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2184 - acc: 0.9276 - val_loss: 0.1265 - val_acc: 0.9607

Epoch 00352: val_loss did not improve from 0.11232
Epoch 353/400
30907/30907 [==============================] - 3s 90us/step - loss: 0.2166 - acc: 0.9285 - val_loss: 0.1126 - val_acc: 0.9677

Epoch 00353: val_loss did not improve from 0.11232
Epoch 354/400
30907/30907 [=============

30907/30907 [==============================] - 3s 91us/step - loss: 0.2018 - acc: 0.9342 - val_loss: 0.1188 - val_acc: 0.9645

Epoch 00391: val_loss did not improve from 0.10366
Epoch 392/400
30907/30907 [==============================] - 3s 90us/step - loss: 0.2031 - acc: 0.9330 - val_loss: 0.0991 - val_acc: 0.9691

Epoch 00392: val_loss improved from 0.10366 to 0.09906, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5
Epoch 393/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2022 - acc: 0.9334 - val_loss: 0.1095 - val_acc: 0.9639

Epoch 00393: val_loss did not improve from 0.09906
Epoch 394/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2047 - acc: 0.9327 - val_loss: 0.1010 - val_acc: 0.9691

Epoch 00394: val_loss did not improve from 0.09906
Epoch 395/400
30907/30907 [==============================] - 3s 91us/step - loss: 0.2008 - acc: 0.9338 - val_loss: 0.1026 - val_acc: 0.9691

Epoch 00395: val_loss did not i

In [106]:
model = load_model(filepath)
test_y_pred = model.predict(test_x)

In [18]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [107]:
print(confusion_matrix(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(classification_report(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(accuracy_score(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
# PPG RR

[[533   2   2   0   0   3   0   0   6   5   3   2   0   0   0]
 [  1 685   0   1   2   0   0   6   0   1   0   3   0   0   1]
 [  2   0 526   3   0   5   0   2   3   0   0   0   1   4   1]
 [  0   0   5 504   0   3   0   1   2   0   0   0   1   2   0]
 [  3   3   0   2 602   1   0   0   1   3   0   2   0   0   2]
 [ 10   2   2   0   0 521   0   4   4   0   0   0   2   3   0]
 [  1   0   1   0   0   0 634   1   1   1   0   0   2   0   2]
 [  0   5   1   1   0   4   2 521   0   1   0   2   0   3   0]
 [  0   0   5   1   2   2   0   2 501   0   0   1   2   0   0]
 [  1   3   0   0   4   0   1   0   0 623   2   2   0   0   0]
 [  1   0   0   1   2   0   1   1   0   2 507   2   0   0   0]
 [  0   4   0   0   0   3   0   2   2   0   0 588   2   0   0]
 [  1   0   1   1   0   1   5   1   3   0   1   3 491   4   5]
 [  1   0   1   1   0   6   0   0   0   0   0   0   1 533   0]
 [  0   5   1   1   1   0   3   1   1   5   4   0   2   7 554]]
              precision    recall  f1-score   support


In [47]:
print(confusion_matrix(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(classification_report(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(accuracy_score(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
# PPG HR Normalized

[[543   0   1   1   0   1   3   0   1   0   0   0   1   4   1]
 [  0 675   0   1   4   0   1   7   0   1   1   8   2   0   0]
 [  1   0 538   0   0   2   0   0   0   0   0   0   2   3   1]
 [  0   0   2 499   1   0   0   6   2   0   1   1   3   3   0]
 [  0   0   0   0 612   0   0   0   2   1   1   1   0   0   2]
 [  5   0   0   0   0 531   1   0   0   1   0   0   3   4   3]
 [  4   0   0   0   0   1 628   0   0   1   4   1   0   1   3]
 [  2   3   1   0   0   0   0 528   0   1   4   1   0   0   0]
 [  0   0   0   1   0   0   1   1 511   0   0   2   0   0   0]
 [  0   2   0   0   1   0   0   1   1 628   0   1   0   1   1]
 [  0   1   0   0   0   0   5   0   1   0 509   0   0   1   0]
 [  0   2   0   0   0   2   0   1   2   1   0 593   0   0   0]
 [  0   0   4   0   0   1   0   0   1   0   0   0 507   4   0]
 [  5   0   2   0   0   1   0   0   0   0   0   0   1 534   0]
 [  0   0   0   0   0   1   2   0   0   0   1   0   0   1 580]]
              precision    recall  f1-score   support


In [93]:
print(confusion_matrix(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(classification_report(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(accuracy_score(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
# ECG RR

[[539   0   5   0   1   4   0   0   1   1   0   1   3   1   0]
 [  1 678   1   2  11   0   0   4   0   1   0   1   1   0   0]
 [  4   0 531   5   2   1   0   0   0   0   0   0   1   3   0]
 [  0   0   3 509   1   3   0   1   1   0   0   0   0   0   0]
 [  0   1   0   0 609   4   0   1   3   0   0   0   1   0   0]
 [  4   0   1   4   2 531   0   1   1   0   0   0   3   1   0]
 [  0   2   1   0   0   0 628   0   0   5   0   0   0   0   7]
 [  0   3   3   3   0   3   0 513   1   0   1   4   3   4   2]
 [  0   1   4   5   0  14   0   0 488   0   1   1   0   2   0]
 [  1   4   0   1   3   0   0   2   0 621   2   1   0   0   1]
 [  3   1   0   0   0   0   0   2   0   0 510   0   0   0   1]
 [  0   5   0   0   0   3   0   2   3   0   1 587   0   0   0]
 [  1   0   2   1   0   1   0   1   0   0   0   1 503   7   0]
 [  0   0   2   0   0   2   0   0   1   0   0   0   5 532   1]
 [  1   0   0   2   0   0   0   0   0   0   0   3   3   0 576]]
              precision    recall  f1-score   support


In [79]:
print(confusion_matrix(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(classification_report(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
print(accuracy_score(np.argmax(test_y,axis=1),np.argmax(test_y_pred,axis=1)))
# ECG RR Normalized

[[552   0   0   0   1   2   0   0   0   0   0   0   0   0   1]
 [  0 694   0   0   2   0   1   0   1   1   0   0   0   1   0]
 [  1   0 539   0   0   0   2   0   0   0   0   1   1   3   0]
 [  0   0   0 516   1   0   0   0   0   0   0   0   0   1   0]
 [  0   0   0   1 611   5   0   1   0   0   1   0   0   0   0]
 [  1   0   0   0   1 545   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0 637   1   0   1   1   0   0   0   3]
 [  0   0   0   0   0   0   0 538   0   1   0   0   0   1   0]
 [  0   1   0   0   1   0   0   2 512   0   0   0   0   0   0]
 [  0   0   0   0   2   0   0   1   0 632   0   0   0   0   1]
 [  0   0   0   0   1   0   0   0   0   0 514   1   0   1   0]
 [  0   4   0   0   0   0   2   0   0   2   1 592   0   0   0]
 [  4   0   0   0   1   0   0   0   0   0   0   1 510   0   1]
 [  1   2   0   0   0   0   0   0   0   0   0   0   0 540   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   1   1 582]]
              precision    recall  f1-score   support
